<a href="https://colab.research.google.com/github/Catisyf/NLP_project_chez_Toucan/blob/main/back_fill_text_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -U sentence-transformers #sentence-transformers model

In [ ]:
pip install Faker #create fake data 

In [ ]:
!ls '/content/drive/My Drive/crm_contacts.csv'

'/content/drive/My Drive/crm_contacts.csv'


In [ ]:
import pandas as pd
import numpy as np
import re
from scipy import spatial
from collections import Counter 
from faker import Faker

In [ ]:
df_contacts = pd.read_csv('/content/drive/My Drive/crm_contacts.csv', na_values=' ') #replace blank cells with na
df_persona = pd.read_csv('/content/drive/My Drive/buyer_persona_definition.csv')

## **Data Preparation**
- Step 1. Remove data points with null values as well as auto-filled job titles. 
- Step 2. Apply pre-defined functions to remove stop words and special characters.

In [ ]:
df_contacts = df_contacts.replace('?', 'np.Nan').dropna() #remove null

In [ ]:
#remove auto-filled job titles
df_contacts = df_contacts[~df_contacts.jobtitle.str.contains('(site)|(try-for-free)')] #add or remove keywords here if necessary

In [ ]:
#create stopwords list
#swtich to stopwords corpus when processing complex text data (check nltk or spaCy)
stopwords = ['of', 'de', 'De', 'du', 'for', 'For', 'the', 'et', "d’"] #add or remove stop words here if neccessary 
space = ' '

In [ ]:
#function to remove stopwords from job title
def remove_stopwords (df_column):
    clean = [space.join([word for word in name.split() \
                         if word not in stopwords]) \
             for name in df_column]
    return clean

In [ ]:
#function to remove special characters from job title
def remove_sign(df_column):
  clean = [re.sub('[|!@#$-.&/_+={}()]', ' ', text) for text in df_column] #add or remove signs here if necessary
  return clean

In [ ]:
#add cleaned job title to datasets
df_persona['clean_title'] = remove_stopwords(df_persona['job_title'])
df_contacts['clean_title'] = remove_stopwords(df_contacts['jobtitle'])

In [ ]:
df_persona['clean_title'] = remove_sign(df_persona['clean_title'])
df_contacts['clean_title'] = remove_sign(df_contacts['clean_title'])

## **NLP model for matching**

In [ ]:
#load pre-trained sentence transformer model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [ ]:
#function to vectorize job title
#use the broadcasting function in numpy array to speed up processing: reduced run time to half
def text_vectorizer (df_column):
  vector = model.encode(df_column.values)
  
  return vector.tolist()

In [ ]:
#add vector to datasets
df_persona['vector'] = text_vectorizer (df_persona['clean_title'])
df_contacts['vector'] = text_vectorizer (df_contacts['clean_title'])

In [ ]:
#reset index 
df_contacts = df_contacts.reset_index(drop = True)

In [ ]:
#function to calculate cosine similarity
def cosine_similarity(v1,v2):
  return 1-spatial.distance.cosine(v1, v2)

In [ ]:
#function to find a match for each job title in CRM data using the max value of cosine similarity 

def get_matches(df_column_1, df_column_2):
  similarity = {}
  i = 0
  for vec1 in df_column_1:
    similarity_persona = [cosine_similarity(vec1, vec2) for vec2 in df_column2]
    
    max_similiarity = max(similarity_persona)
    persona_id = similarity_persona.index(max_similiarity) 
    similarity[i] = [max_similiarity, persona_id]
    i += 1
  
  return similarity

In [ ]:
table = pd.DataFrame.from_dict(get_matches(df_contacts['vector'], df_persona['vector']), orient = 'index')
table.columns =['similarity_score', 'persona_job_index']

In [ ]:
#join tables using index 
table_full = table.join(df_contacts[['jobtitle', 'email', 'vector']]\
                       ).merge(df_persona[['buyer_persona', 'job_title', 'vector']], \
                               left_on = 'persona_job_index', right_index = True)

In [ ]:
#rename table column
table_full = table_full.rename(columns={'jobtitle': "job_title_CRM", 'job_title': 'job_title_toucan'})

In [ ]:
#select useful columns
table_full = table_full[['job_title_CRM', 'email', 'job_title_toucan', 'buyer_persona', 'similarity_score']]

#filter high similarity score to get good matches 
table_good_score = table_full.query('similarity_score >= 0.85')

#query filtered data
table_filtered_results = table_full.query('similarity_score < 0.85')

In [ ]:
#create fake emails to anonymise confidential data
faker = Faker()
fake_email = [faker.company_email() for i in range(349)]
table_good_score['email'] = fake_email

In [ ]:
table_good_score

,job_title_CRM,email,job_title_toucan,buyer_persona,similarity_score
4146,Founders,mnunez@woodward.com,Founder,Business Leader,0.902885
24,Manager of Sales,kiaracannon@thompson.com,Sales Director,Business Leader,0.932112
812,Sales Executive,amanda38@greene-lopez.net,Sales Director,Business Leader,0.917183
854,Vendor Manager,yatesanthony@cunningham-hood.com,Sales Director,Business Leader,0.920250
1907,Sales Manager,kelly66@boone-maldonado.com,Sales Director,Business Leader,0.936844
...,...,...,...,...,...
3241,Consultor de Business Intelligence,ericreyes@anderson.com,Consultant Business Intelligence,Practitioner,0.986818
5007,Expert Business Intelligence (Maltem Consulting),tinamosley@rivers.info,Consultant Business Intelligence,Practitioner,0.869026
12371,Consultante Business Intelligence,robertsuarez@taylor.com,Consultant Business Intelligence,Practitioner,0.999221
16493,Consultante Pilotage & Business Intelligencea,rasmussendanny@waller.com,Consultant Business Intelligence,Practitioner,0.896643
